
# Frage 2 – Notebook 02: Aufbereitung & Normalisierung

Dieses Notebook:
- vereinheitlicht die Kategorien der Online-Zahlungsarten (2019/2021/2023),
- führt die Jahre zusammen,
- bereitet die Umfrage (Mehrfachantworten) auf,
- persistiert normalisierte Tabellen als CSV.


## Pfade & Imports

In [1]:

from pathlib import Path
import pandas as pd
import re

# Projekt-Root ermitteln (ggf. anpassen)
project_root = Path("D:/JupyterProject")
data_path = project_root / "data"
out = project_root / "Frage2"
out.mkdir(exist_ok=True, parents=True)

pd.set_option("display.width", 140)


## 1) Roh-CSV laden

In [2]:

df2019 = pd.read_csv(out / "online2019_raw.csv")
df2021 = pd.read_csv(out / "online2021_raw.csv")
df2023 = pd.read_csv(out / "online2023_raw.csv")
stationär = pd.read_csv(out / "stationaer_raw.csv")
umf = pd.read_csv(out / "umfrage_raw.csv")


## 2) Normalisierungs-Mappings

In [3]:

norm23 = {
    "E-Wallet (Paypal, Alipay)": "E-Wallet",
    "Per Rechnung (und Zahlschein)": "Rechnung",
    "Klarna": "BNPL (Klarna)",
    "Kreditkarte einer inländischen Bank / Debitkarte": "Kreditkarte",
    "Visa / Mastercard": "Kreditkarte",
    "Banküberweisung": "Überweisung/Online-Transfer",
    "Direct debit": "Lastschrift/SEPA",
    "Sofort": "Überweisung/Online-Transfer",
    "Mobile Bezahl-App": "Mobile Wallet",
    "Virtuelle Währungen (Bitcoin)": "Krypto"
}
norm21 = {
    "E-Wallets": "E-Wallet",
    "Auf Rechnung": "Rechnung",
    "Bezahlung mit Kreditkarte": "Kreditkarte",
    "SEPA-Direktmandat": "Lastschrift/SEPA",
    "Online-Transfer": "Überweisung/Online-Transfer",
    "Mobile Geldbörsen": "Mobile Wallet",
    "Lieferung per Nachnahme": "Nachnahme",
}
norm19 = {
    "Über einen Bezahldienstleister (z.B. Paypal)": "E-Wallet",
    "Rechnung": "Rechnung",
    "Kreditkarte": "Kreditkarte",
    "Per Überweisung": "Überweisung/Online-Transfer",
    "Per Lastschrift": "Lastschrift/SEPA",
    "Sofortüberweisung": "Überweisung/Online-Transfer",
    "Vorkasse": "Nachnahme"
}

def normalize(df, mapping):
    d = df.copy()
    d["Methode"] = d["Methode"].astype(str).str.strip()
    d["Kategorie"] = d["Methode"].map(mapping).fillna(d["Methode"])
    d["Prozent"] = pd.to_numeric(d["Prozent"], errors="coerce")
    return d.groupby("Kategorie", as_index=False)["Prozent"].sum()

n19 = normalize(df2019, norm19).rename(columns={"Prozent":"pct_2019"})
n21 = normalize(df2021, norm21).rename(columns={"Prozent":"pct_2021"})
n23 = normalize(df2023, norm23).rename(columns={"Prozent":"pct_2023"})

online_merged = n23.merge(n21, on="Kategorie", how="outer").merge(n19, on="Kategorie", how="outer")
online_merged.to_csv(out / "online_normalisiert.csv", index=False)
online_merged.sort_values("pct_2023", ascending=False).head(10)


,Kategorie,pct_2023,pct_2021,pct_2019
5,E-Wallet,65.0,39.0,42.1
8,Kreditkarte,54.0,10.0,10.1
13,Rechnung,43.0,23.0,28.5
15,Überweisung/Online-Transfer,34.0,6.0,11.3
2,BNPL (Klarna),28.0,NaN,NaN
10,Lastschrift/SEPA,20.0,8.0,5.7
14,Voucher ePay / Geschenkkarte eines Geschäfts /...,18.0,NaN,NaN
6,Giropay,9.0,NaN,NaN
3,Bezahlung im Geschäft,9.0,NaN,NaN
7,Kauf auf Kredit,8.0,NaN,NaN


## 3) Umfrage normalisieren (AP/AQ)

In [4]:

def clean_text(s):
    if pd.isna(s): return ""
    return str(s).replace("\xa0", " ").replace("„","").replace("“","").strip()

def norm_method(token):
    t = clean_text(token)
    t = re.sub(r"buy\s*now\s*pay\s*later.*", "BNPL (Klarna)", t, flags=re.I)
    t = re.sub(r"krypto.*", "Krypto", t, flags=re.I)
    mapping = {
        "paypal": "E-Wallet",
        "kreditkarte": "Kreditkarte",
        "rechnung": "Rechnung",
        "lastschrift": "Lastschrift/SEPA",
        "sofortüberweisung": "Überweisung/Online-Transfer",
        "apple pay": "Mobile Wallet",
        "bnpl (klarna)": "BNPL (Klarna)",
    }
    return mapping.get(t.lower(), t)

# Zahlungsarten-Mehrfachauswahl
methods = (umf["Zahlungsarten"].astype(str).map(clean_text).str.split(";", expand=False)
           .explode().dropna().map(norm_method).str.strip())
methods = methods[methods.ne("")]
umfrage_counts = methods.value_counts().rename_axis("Kategorie").reset_index(name="Anzahl")
umfrage_counts["pct_umfrage"] = (umfrage_counts["Anzahl"] / len(umf) * 100).round(1)

# BNPL-Veränderung
bnpl = (umf["BNPL_Veraenderung"].fillna("").astype(str).str.strip())
bnpl_counts = (bnpl.replace({"Ich habe BNPL noch nie genutzt":"Nie",
                             "Ich nutze BNPL häufiger als früher":"Häufiger",
                             "Ich nutze BNPL etwa gleich häufig":"Gleich",
                             "Ich nutze BNPL seltener als früher":"Seltener"})
                    .value_counts().rename_axis("Antwort").reset_index(name="Anzahl"))

umfrage_counts.to_csv(out / "umfrage_methods_norm.csv", index=False)
bnpl_counts.to_csv(out / "umfrage_bnpl_counts.csv", index=False)

umfrage_counts.head(), bnpl_counts.head()


(          Kategorie  Anzahl  pct_umfrage
 0          E-Wallet      32         82.1
 1       Kreditkarte      16         41.0
 2  Lastschrift/SEPA      13         33.3
 3          Rechnung      13         33.3
 4     BNPL (Klarna)       6         15.4,
     Antwort  Anzahl
 0       Nie      28
 1  Häufiger       5
 2  Seltener       4
 3    Gleich       2)